# Prueba chill

In [11]:
import torch
import numpy as np
import cv2
from facenet_pytorch import MTCNN, InceptionResnetV1
from sklearn.metrics.pairwise import cosine_similarity
import os

# Inicialización
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(image_size=160, margin=20, keep_all=False, device=device)
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Función para obtener embedding de una imagen
def get_embedding(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    face = mtcnn(img)
    if face is None:
        raise ValueError(f"No se detectó rostro en la imagen {img_path}")
    face = face.unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model(face).cpu().numpy()
    embedding = embedding / np.linalg.norm(embedding)  # Normalizar
    return embedding

# Crear base de datos de embeddings a partir de carpeta con imágenes de personas
def create_database(folder_path):
    database = {}
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            person_name = os.path.splitext(filename)[0]
            img_path = os.path.join(folder_path, filename)
            try:
                emb = get_embedding(img_path)
                database[person_name] = emb
                print(f"Embedding creado para {person_name}")
            except Exception as e:
                print(f"Error con {filename}: {e}")
    return database

# Función para reconocer persona en imagen de prueba
def recognize_face(img_path, database, threshold=0.5):
    emb = get_embedding(img_path)
    max_sim = -1
    identity = "Desconocido"

    for name, db_emb in database.items():
        sim = cosine_similarity(emb, db_emb)[0][0]  # Similaridad coseno
        if sim > max_sim:
            max_sim = sim
            identity = name

    if max_sim < threshold:
        identity = "Desconocido"

    return identity, max_sim

def main():
    # Ruta a carpeta con imágenes de personas conocidas
    known_folder = 'personas_conocidas'
    database = create_database(known_folder)

    # Imagen a reconocer
    test_image = 'jimmy.jpg'
    identidad, similitud = recognize_face(test_image, database)

    print(f"Identidad detectada: {identidad} con similitud {similitud:.3f}")

if __name__ == "__main__":
    main()


Embedding creado para Cristian
Embedding creado para Dan Peralta
Embedding creado para Jimmy
Identidad detectada: Jimmy con similitud 0.895


# Ver con cámara

In [4]:
import torch
import numpy as np
import cv2
from facenet_pytorch import MTCNN, InceptionResnetV1
from sklearn.metrics.pairwise import cosine_similarity

# Selección automática de dispositivo (GPU o CPU)
device = torch.device( 'cpu')

# Inicialización de MTCNN para detección facial
mtcnn = MTCNN(image_size=160, margin=20, keep_all=True, device=device)

# Modelo para extracción de embeddings faciales
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

def get_embedding(face_img):
    """
    Recibe tensor [3,160,160] con imagen RGB normalizada.
    Retorna embedding facial normalizado.
    """
    face_img = face_img.unsqueeze(0).to(device)  # Añadir batch y enviar a device
    with torch.no_grad():
        embedding = model(face_img).cpu().numpy()
    embedding = embedding / np.linalg.norm(embedding)
    return embedding

def recognize_face(embedding, database, threshold=0.5):
    """
    Compara embedding con base de datos y devuelve nombre e índice de similitud.
    """
    max_sim = -1
    identity = "Desconocido"
    for name, db_emb in database.items():
        sim = cosine_similarity(embedding, db_emb)[0][0]
        if sim > max_sim:
            max_sim = sim
            identity = name
    if max_sim < threshold:
        identity = "Desconocido"
    return identity, max_sim

def main():
    # Cargar base de datos guardada
    database = np.load('database_embeddings.npy', allow_pickle=True).item()
    print(f"Base cargada con {len(database)} personas.")

    cap = cv2.VideoCapture(0)  # Ajusta índice según cámara (0,1,...)
    if not cap.isOpened():
        print("No se pudo abrir la cámara.")
        return

    print("Reconocimiento facial en tiempo real. Presiona 'q' para salir.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error al capturar frame de cámara.")
            break

        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        boxes, _ = mtcnn.detect(img_rgb)

        if boxes is not None:
            h, w, _ = img_rgb.shape
            for box in boxes:
                try:
                    # Limitar coordenadas dentro de la imagen
                    x1, y1, x2, y2 = [int(b) for b in box]
                    x1, y1 = max(0, x1), max(0, y1)
                    x2, y2 = min(w, x2), min(h, y2)
                    if x2 <= x1 or y2 <= y1:
                        continue  # bounding box inválido, saltar

                    # Recortar y preparar imagen para embedding
                    face_img = img_rgb[y1:y2, x1:x2]
                    face_img = cv2.resize(face_img, (160, 160))
                    face_tensor = torch.tensor(face_img).permute(2, 0, 1).float() / 255.0

                    # Obtener embedding y reconocer persona
                    embedding = get_embedding(face_tensor)
                    name, similarity = recognize_face(embedding, database)

                    # Dibujar rectángulo y texto en imagen original
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"{name} ({similarity:.2f})", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                except Exception as e:
                    print(f"Error procesando rostro: {e}")
                    continue

        cv2.imshow('Reconocimiento Facial', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()



Base cargada con 13 personas.
Reconocimiento facial en tiempo real. Presiona 'q' para salir.


# Generar DataBase

In [3]:
import os
import torch
import numpy as np
import cv2
from facenet_pytorch import MTCNN, InceptionResnetV1

# Selección automática de dispositivo (GPU o CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Inicialización de MTCNN para detección facial
mtcnn = MTCNN(image_size=160, margin=20, keep_all=False, device=device)  # keep_all=False porque asumimos una cara por imagen

# Modelo para extraer los embebbings
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

def get_embedding(face_img):
    """
    Recibe tensor [3,160,160] con imagen RGB normalizada.
    Retorna embedding facial normalizado (numpy array).
    """
    face_img = face_img.unsqueeze(0).to(device)  # Añadir batch y enviar a device
    with torch.no_grad():
        embedding = model(face_img).cpu().numpy()
    embedding = embedding / np.linalg.norm(embedding)
    return embedding

def load_image(path):
    img = cv2.imread(path)
    if img is None:
        raise ValueError(f"No se pudo leer la imagen {path}")
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img_rgb

def process_person_images(folder_path):
    database = {}
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            path = os.path.join(folder_path, filename)
            print(f"Procesando {filename}...")
            img_rgb = load_image(path)

            # Detectar la cara (solo una por imagen)
            face_tensor = mtcnn(img_rgb)
            if face_tensor is None:
                print(f"No se detectó cara en {filename}. Se omitirá.")
                continue

            # Obtener embedding 
            embedding = get_embedding(face_tensor)

            # Usar el nombre de archivo sin extensión como clave (puedes modificar según estructura)
            person_name = os.path.splitext(filename)[0]
            database[person_name] = embedding

    return database

if __name__ == "__main__":
    folder = "personas_conocidas"
    database_embeddings = process_person_images(folder)

    # Guardar la base de datos de embeddings
    np.save('database_embeddings.npy', database_embeddings)
    print(f"Base de datos con {len(database_embeddings)} embeddings guardada en 'database_embeddings.npy'.")


Procesando Alan.jpg...
Procesando Alvaro.jpg...
Procesando Cristian.jpg...
Procesando Dan Peralta.jpg...
Procesando Emma.jpg...
Procesando Jenifer.jpg...
Procesando Jimmy.jpg...
Procesando Loli.jpg...
Procesando Lucy.jpg...
Procesando Manola.jpg...
Procesando Melany.jpg...
Procesando Sasha.jpg...
Procesando Sergio.jpg...
Base de datos con 13 embeddings guardada en 'database_embeddings.npy'.
